This code is based on Python 3.5.

I propose only the code version without chunck (the whole file is processed in a single step).

Using chunks only concerns the generation of "dfGroupDestMonth" which can be coded as in Exercise 2. The dataframe "dfGroupDestMonth" has a reasonable size.


In [1]:
# common parameters of the code
import pandas as pd 
import matplotlib.pyplot as plt
# to insert plot within the notebook
%matplotlib inline 

listTargetDestination=["MAD","BCN","AGP"] # List of target destinations: MADRID (MAD), BARCELONA (BCN) and MALAGA (AGP)
usedColumns = ['Date','Destination'] # column relevant for the processing
listMonthNumber=[1,2,3,4,5,6,7,8,9,10,11,12] # list of month numbers

I read the file and declare NaT (option errors='coerce') the dates uncorrectly formatted.

The correct format is '%Y-%m-%d'

In [2]:
parse = lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors='coerce')
df = pd.read_csv("searches.csv",sep='^', usecols=usedColumns, parse_dates=['Date'],date_parser=parse)


I count the number of searches per destination and per month.

The final result is a dataframe which is indexed by the month number between 1 and 12.

In [3]:
dfGroupDestMonth=df['Date'].groupby([df.Destination, df.Date.dt.month]).count().reset_index().set_index('Date')

# I count the number of searches for each target destination.
# A loop could be used but it seems useless since I have only three target destinations
# I reindex the dataframe in order to complete the months if one month has a null count 
# (in this case, the month does not appear in the dataframe 'dfGroupDestMonth')
# The value 0 is assigned to missing months (option "fill_value=0")
dfcount0 = dfGroupDestMonth[dfGroupDestMonth.Destination==listTargetDestination[0]].reindex(
    listMonthNumber, fill_value=0)# MAD
dfcount1 = dfGroupDestMonth[dfGroupDestMonth.Destination==listTargetDestination[1]].reindex(
    listMonthNumber, fill_value=0)# BCN
dfcount2 = dfGroupDestMonth[dfGroupDestMonth.Destination==listTargetDestination[2]].reindex(
    listMonthNumber, fill_value=0)# AGP

# I merge all the dataframes to obtain one final dataframe whose columns contain the 
# number of searches for each target destination
dftmp = pd.merge(dfcount0,dfcount1,left_index=True, right_index=True) # merge of two dataframes with indexes
dftmp = pd.merge(dftmp,dfcount2,left_index=True, right_index=True) # merge of two dataframes with indexes
dfDestinationCounts = dftmp.iloc[:,[1,3,5]] # This is the final dataframe


Plot the result: coming soon

I just print the dataframe for the moment without renaming the columns

In [6]:
print(dfDestinationCounts)

        0_x    0_y      0
Date                     
1     24258  29469   9633
2     22800  28329   8379
3     24681  30552  10659
4     25251  31236   8265
5     26334  28728  10830
6     22800  26505   7923
7     22971  29241   8892
8     21831  27075   7866
9     21147  23427   8151
10    22294  20276   6499
11    20272  19824   6384
12    14504  15400   3696
